# OpenAI OSS Model Getting Started Guide on Amazon Bedrock

This notebook provides a comprehensive introduction to using gpt-oss-120b & gpt-oss-20b on Amazon Bedrock, including how to leverage the familiar OpenAI SDK interface with Amazon Bedrock. We'll cover how to make API requests, explore available parameters and payload structures, and examine use cases for both model variants. 

## Model Variants

### GPT-OSS-120b

Parameters: 120 billion

Use Cases: Complex reasoning tasks, agentic use cases

### GPT-OSS-20b

Parameters: 20 billion

Use Cases: Faster inference, cost-effective deployments, simpler tasks

## Core Capabilities

Both OpenAI model variants share the following characteristics:

**Input/Output:** Text-in, text-out 

**Context Window:** 128,000 tokens  

**Model Type:** Advanced reasoning models

**Region:** us-west-2

**Tool Calling:** ✅ Supported

**Bedrock Guardrails** ✅ Supported

**Converse API** ✅ Supported

**OpenAI Chat Completions API** ✅ Supported

**Web Search:** ❌ Not available at this time

## What You'll Learn in this getting started guide

- Options to use Amazon Bedrock for GPT-OSS inference, including:    
    - Using the OpenAI SDK with Amazon Bedrock
    - Using Amazon Bedrock's InvokeModel API
    - Using Amazon Bedrock's Converse API
- Understanding request parameters and response structures
- Choosing between Large and Small models for your use case
- Implementing tool calling capabilities

## Model Access on Amazon Bedrock

Ensure you have the correct IAM permission in order to access OpenAI's models on Amazon Bedrock. In order to ensure you have model access, follow these steps: 

- Go to Amazon Bedrock --> model access
- Click Modify model access
- Scroll to OpenAI
- Click the checkbox next to the models you would like access to
- Click next & accept any EULAs
- Click submit

## IAM Permissions

To use Bedrock models, your AWS credentials need the following permissions:


In [ ]:
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "bedrock:InvokeModel",
        "bedrock:InvokeModelWithResponseStream"
      ],
      "Resource": "*"
    }
  ]
}


## Step 1: Environment Configuration

First, we need to install the required packages and tell the OpenAI SDK to talk to Bedrock instead of OpenAI's servers.

### Required Imports:
- `os` → For environment variables
- `boto3` → For native Bedrock API interactions  
- `json` → For JSON serialization/deserialization
- `datetime` → For timestamp tracking and performance measurements
- `openai` → For OpenAI SDK compatibility with Bedrock
- `strands` → For Amazon Strands agent framework
- `IPython.display` → For enhanced output formatting and streaming demonstrations

### Environment Variables:
We set two environment variables to redirect the OpenAI SDK:
- `AWS_BEARER_TOKEN_BEDROCK` → Your Bedrock API key  
- `OPENAI_BASE_URL` → Bedrock's OpenAI-compatible endpoint

In [ ]:
%pip -U install boto3 openai strands-agents ipython

In [ ]:
import os
import boto3
import json
from openai import OpenAI
from strands import Agent
from strands.models.openai import OpenAIModel
from strands.models import BedrockModel
from datetime import datetime
from IPython.display import clear_output, display, display_markdown, Markdown

### Model IDs

- **openai.gpt-oss-120b-1:0**
- **openai.gpt-oss-20b-1:0**

In [ ]:
# Model Configuration - Change this to your desired model
MODEL_ID = "gpt-oss-120b"  

print(f"✅ Using model: {MODEL_ID}")

In [ ]:
# Set environment variables to point to Bedrock
os.environ["AWS_BEARER_TOKEN_BEDROCK"] = "<insert your bedrock API key>"
os.environ["OPENAI_BASE_URL"] = "https://bedrock-runtime.us-west-2.amazonaws.com/openai/v1"

print("✅ Environment configured for Bedrock!")

## Step 2: Inference with Amazon Bedrock

### Option 1: OpenAI SDK

#### Import and Initialize OpenAI Client

Now we use the **exact same OpenAI SDK** you're familiar with. The client will automatically read the environment variables we just set.

**Key Point**: This is the same OpenAI library, but now it's talking to Amazon Bedrock.

In [ ]:
# Initialize both clients
client = OpenAI()  # For chat completions API
bedrock_client = boto3.client('bedrock-runtime', region_name='us-west-2')  

print("✅ OpenAI client initialized (pointing to Bedrock)")
print(f"✅ Bedrock client initialized in region: {bedrock_client.meta.region_name}")

#### Make API Calls 

The API call structure is identical to OpenAI:
- Same `messages` format with `role` and `content`
- Same `model` parameter (but uses Bedrock model IDs)  
- Same `stream` parameter for real-time responses

In [ ]:
response = client.chat.completions.create(
    model="gpt-oss-120b",                 
    messages=[
        {"role": "system", "content": "You are a concise, highly logical assistant."},
        {"role": "user",   "content": "What is the largest city in the southern hemisphere?"}
    ],
    temperature=0.2,
    max_tokens=1000,
    stream=True                         
)

print("✅ API request created")


#### Process Streaming Response

Handle the response exactly like you would with OpenAI. Each `item` in the response is a chunk of the model's output.

In [ ]:
for item in response:
    print(item)

#### What's Happening Behind the Scenes?

When you use the OpenAI SDK with Bedrock, your requests are automatically translated to Bedrock's native `InvokeModel` API.

#### Request Translation
- **OpenAI SDK Request** → **Bedrock InvokeModel** 
- The request body structure remains the same
- But there are some key differences in how parameters are handled:

| Parameter | OpenAI SDK | Bedrock InvokeModel |
|-----------|------------|-------------------|
| **Model ID** | In request body | Part of the URL path |
| **Streaming** | `stream=True/False` | Different API endpoints:<br/>• `InvokeModel` (non-streaming)<br/>• `InvokeModelWithResponseStream` (streaming) |
| **Request Body** | Full chat completions format | Same format, but `model` and `stream` are optional |

#### Function Calling with OpenAI SDK

The OpenAI SDK interface on Bedrock supports function calling, allowing the model to invoke external functions. Let's demonstrate this with a weather lookup function.


In [ ]:
def get_weather(location):
    """
    Get current weather for a given location.
    This is a mock function that returns sample weather data.
    
    Args:
        location (str): City and country, e.g. "Paris, France"
        
    Returns:
        dict: Weather information
    """
    # Mock weather data - in a real application, you'd call a weather API
    weather_data = {
        "Paris, France": {"temperature": "22°C", "condition": "Partly cloudy", "humidity": "65%"},
        "New York, USA": {"temperature": "18°C", "condition": "Sunny", "humidity": "45%"},
        "Tokyo, Japan": {"temperature": "25°C", "condition": "Rainy", "humidity": "80%"},
        "London, UK": {"temperature": "15°C", "condition": "Overcast", "humidity": "70%"},
        "Sydney, Australia": {"temperature": "28°C", "condition": "Clear", "humidity": "55%"}
    }
    
    return weather_data.get(location, {
        "temperature": "20°C", 
        "condition": "Data not available", 
        "humidity": "50%"
    })

# Define the function schema for OpenAI SDK
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature and weather conditions for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country, e.g. 'Paris, France'"
                }
            },
            "required": ["location"],
            "additionalProperties": False
        }
    }
}]

print("✅ Weather function and tools configuration ready!")


In [ ]:
def chat_with_functions(client, model, messages, tools, max_iterations=3):
    """
    Chat with function calling support using OpenAI SDK format.
    
    Args:
        client: OpenAI client instance
        model: Model ID to use
        messages: List of conversation messages
        tools: List of available tools/functions
        max_iterations: Maximum number of function call iterations
        
    Returns:
        Final assistant message
    """
    
    for iteration in range(max_iterations):
        print(f"🔄 Iteration {iteration + 1}")
        
        # Make request with tools
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        assistant_message = response.choices[0].message
        messages.append(assistant_message)
        
        # Check if the model wants to call functions
        if assistant_message.tool_calls:
            print(f"🔧 Model requested {len(assistant_message.tool_calls)} function call(s)")
            
            # Process each function call
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"🔧 Calling function: {function_name}")
                print(f"🔧 Arguments: {function_args}")
                
                # Call the actual function
                if function_name == "get_weather":
                    function_result = get_weather(function_args["location"])
                    print(f"🔧 Function result: {function_result}")
                else:
                    function_result = {"error": f"Unknown function: {function_name}"}
                
                # Add function result to conversation
                function_message = {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps(function_result)
                }
                messages.append(function_message)
                
        else:
            # No more function calls, return final response
            print("✅ No function calls requested, conversation complete")
            return assistant_message
    
    print("⚠️ Maximum iterations reached")
    return assistant_message

print("✅ Function calling handler ready!")


In [ ]:
# Test function calling with various weather queries
weather_questions = [
    "What's the weather like in Paris today?",
    "Can you tell me the temperature in Tokyo?",
    "How's the weather in Sydney, Australia?",
    "What are the conditions like in New York?"
]

print("🌤️ Testing Function Calling with OpenAI SDK")
print("=" * 50)

for i, question in enumerate(weather_questions, 1):
    print(f"\n📝 Test {i}: {question}")
    print("-" * 40)
    
    try:
        # Create conversation messages
        messages = [
            {"role": "system", "content": "You are a helpful weather assistant. Use the get_weather function to provide accurate weather information."},
            {"role": "user", "content": question}
        ]
        
        # Call the function calling handler
        final_response = chat_with_functions(
            client=client,
            model=MODEL_ID,  # Uses the configured model ID
            messages=messages,
            tools=tools
        )
        
        # Print the final response
        print("🤖 Final response:")
        print(final_response.content)
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
    
    print()


#### What Just Happened with Function Calling?

The function calling demonstration above shows the OpenAI SDK workflow:

1. **Function Definition**: We defined a weather function with OpenAI SDK schema format
2. **Initial Request**: The model receives a weather question and recognizes it needs weather data
3. **Function Call**: The model generates a structured function call request with proper arguments
4. **Function Execution**: Our handler extracts the arguments, calls the actual function, and gets results
5. **Result Integration**: The function result is added back to the conversation context
6. **Final Response**: The model uses the function result to provide a natural language answer



### Option 2: Amazon Bedrock's InvokeModel API

The Bedrock InvokeModel API is the foundational interface for interacting directly with any model hosted on Amazon Bedrock. It provides low-level, flexible access to model inference, allowing you to send input data and receive generated responses in a consistent way across all supported models.

**Key Benefits:**
- Direct Access: Interact with any Bedrock model using a unified API endpoint.
- Fine-Grained Control: Customize inference parameters and payloads for each request.
- Streaming Support: Use `InvokeModelWithResponseStream` for real-time, token-by-token output.
- Privacy: Amazon Bedrock does not store your input or output data—requests are used only for inference.

#### Setup client

First, we setup the Amazon Bedrock client. 

In [ ]:
region = None

if region is None:
    target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
else:
    target_region = "us-west-2"

bedrock_runtime = boto3.client('bedrock-runtime', region_name=region)

#### Inference with InvokeModel API

Then we use the InvokeModel API to perform model inference. 

In [ ]:
def invoke_model(body, model_id, accept, content_type):
    """
    Invokes Amazon bedrock model to run an inference
    using the input provided in the request body.
    
    Args:
        body (dict): The invokation body to send to bedrock
        model_id (str): the model to query
        accept (str): input accept type
        content_type (str): content type
    Returns:
        Inference response from the model.
    """

    try:
        response = bedrock_runtime.invoke_model(
            body=json.dumps(body), 
            modelId=model_id, 
            accept=accept, 
            contentType=content_type
        )

        return response

    except Exception as e:
        print(f"Couldn't invoke {model_id}")
        raise e

In [ ]:
messages = [
            {"role": "system", "content": "You are a concise, highly logical assistant."},
            {"role": "user",   "content": "What is the largest city in the southern hemisphere?"}
            ]
  
body = {
    "prompt": messages,
    "temperature": 0.2,
    "max_gen_len": 1000,
}

modelId = "gpt-oss-120b"
accept = "application/json"
contentType = "application/json"

response = invoke_model(body, modelId, accept, contentType)
response_body = json.loads(response.get("body").read())

print(response_body["generation"])

#### Streaming with InvokeModel API

The InvokeModel API comes with built in streaming support. This can be useful in user-facing applications since it reduces time to first token (TTFT) metric and with that perceived inference latency for the end user. 

In [ ]:
messages = [
            {"role": "system", "content": "You are a concise, highly logical assistant."},
            {"role": "user",   "content": "What is the largest city in the southern hemisphere?"}
            ]

body = {
    "prompt": messages,
    "temperature": 0.2,
    "max_gen_len": 1000,
}

modelId = "gpt-oss-120b"
accept = "application/json"
contentType = "application/json"

start_time = datetime.now()

response = bedrock_runtime.invoke_model_with_response_stream(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
chunk_count = 0
time_to_first_token = None

# Process the response stream
stream = response.get("body")
if stream:
    for event in stream:
        chunk = event.get("chunk")
        if chunk:
            # Print the response chunk
            chunk_json = json.loads(chunk.get("bytes").decode())
            # Pretty print JSON
            # print(json.dumps(chunk_json, indent=2, ensure_ascii=False))
            content_block_delta = chunk_json.get("contentBlockDelta")
            if content_block_delta:
                if time_to_first_token is None:
                    time_to_first_token = datetime.now() - start_time
                    print(f"Time to first token: {time_to_first_token}")

                chunk_count += 1
                current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S:%f")
                # print(f"{current_time} - ", end="")
                print(content_block_delta.get("delta").get("text"), end="")
    print(f"Total chunks: {chunk_count}")
else:
    print("No response stream received.")

#### Tool Use with InvokeModel API

The InvokeModel API also supports tool calling through the native Bedrock payload format. Let's demonstrate this with the same weather lookup function to compare approaches across all three APIs.


In [ ]:
def get_weather_invoke(location):
    """
    Get current weather for a given location.
    Same function as used in other API examples for consistency.
    
    Args:
        location (str): City and country, e.g. "Paris, France"
        
    Returns:
        dict: Weather information
    """
    # Same weather data as other examples
    weather_data = {
        "Paris, France": {"temperature": "22°C", "condition": "Partly cloudy", "humidity": "65%"},
        "New York, USA": {"temperature": "18°C", "condition": "Sunny", "humidity": "45%"},
        "Tokyo, Japan": {"temperature": "25°C", "condition": "Rainy", "humidity": "80%"},
        "London, UK": {"temperature": "15°C", "condition": "Overcast", "humidity": "70%"},
        "Sydney, Australia": {"temperature": "28°C", "condition": "Clear", "humidity": "55%"}
    }
    
    return weather_data.get(location, {
        "temperature": "20°C", 
        "condition": "Data not available", 
        "humidity": "50%"
    })

# Tool configuration for InvokeModel API
invoke_model_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature and weather conditions for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country, e.g. 'Paris, France'"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

print("✅ Weather function and InvokeModel API tool configuration ready!")


In [ ]:
def invoke_model_with_tools(bedrock_client, model_id, messages, tools, max_iterations=3):
    """
    Generate text using InvokeModel API with tool support.
    
    Args:
        bedrock_client: Boto3 Bedrock runtime client
        model_id: Model ID to use
        messages: List of conversation messages  
        tools: List of available tools
        max_iterations: Maximum tool calling iterations
        
    Returns:
        Final response from the model
    """
    
    for iteration in range(max_iterations):
        print(f"🔄 Iteration {iteration + 1}")
        
        # Prepare the request body for InvokeModel
        body = {
            "messages": messages,
            "tools": tools,
            "temperature": 0.2,
            "max_tokens": 1000
        }
        
        # Make the InvokeModel request
        response = bedrock_client.invoke_model(
            body=json.dumps(body),
            modelId=model_id,
            accept="application/json",
            contentType="application/json"
        )
        
        # Parse the response
        response_body = json.loads(response.get("body").read())
        assistant_message = response_body.get("choices", [{}])[0].get("message", {})
        
        # Add assistant message to conversation
        messages.append(assistant_message)
        
        # Check if model wants to call tools
        tool_calls = assistant_message.get("tool_calls", [])
        if tool_calls:
            print(f"🔧 Model requested {len(tool_calls)} tool call(s)")
            
            # Process each tool call
            for tool_call in tool_calls:
                function_name = tool_call["function"]["name"]
                function_args = json.loads(tool_call["function"]["arguments"])
                
                print(f"🔧 Calling function: {function_name}")
                print(f"🔧 Arguments: {function_args}")
                
                # Call the actual function
                if function_name == "get_weather":
                    function_result = get_weather_invoke(function_args["location"])
                    print(f"🔧 Function result: {function_result}")
                else:
                    function_result = {"error": f"Unknown function: {function_name}"}
                
                # Add function result to conversation
                function_message = {
                    "tool_call_id": tool_call["id"],
                    "role": "tool", 
                    "name": function_name,
                    "content": json.dumps(function_result)
                }
                messages.append(function_message)
                
        else:
            # No tool calls, return final response
            print("✅ No tool calls requested, conversation complete")
            return assistant_message
    
    print("⚠️ Maximum iterations reached")
    return assistant_message

print("✅ InvokeModel tool calling handler ready!")


In [ ]:
# Test the weather tool with InvokeModel API
weather_questions = [
    "What's the weather like in Paris today?",
    "Can you tell me the temperature in Tokyo?", 
    "How's the weather in Sydney, Australia?",
    "What are the conditions like in New York?"
]

print("🌤️ Testing Weather Tool with InvokeModel API")
print("=" * 50)

for i, question in enumerate(weather_questions, 1):
    print(f"\n📝 Test {i}: {question}")
    print("-" * 40)
    
    try:
        # Create conversation messages
        messages = [
            {"role": "system", "content": "You are a helpful weather assistant. Use the get_weather function to provide accurate weather information."},
            {"role": "user", "content": question}
        ]
        
        # Call the InvokeModel tool handler
        final_response = invoke_model_with_tools(
            bedrock_client=bedrock_runtime,
            model_id="gpt-oss-120b",  # You can change to gpt-oss-20b for faster inference
            messages=messages,
            tools=invoke_model_tools
        )
        
        # Print the final response
        print("🤖 Final response:")
        print(final_response.get("content", "No content in response"))
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
    
    print()


#### What Just Happened with InvokeModel Tool Use?

The InvokeModel API tool use demonstration shows the raw, low-level approach:

1. **Tool Definition**: We defined tools using OpenAI SDK compatible format in the request body
2. **Request Construction**: We manually built the full request payload including messages and tools
3. **API Call**: Direct InvokeModel call with JSON body containing all parameters
4. **Response Parsing**: Manual parsing of the JSON response to extract assistant messages
5. **Tool Detection**: Checking for `tool_calls` in the assistant message
6. **Function Execution**: Extracting arguments, calling functions, and handling results
7. **Conversation Management**: Manually appending messages to maintain conversation context



### Option 3: Amazon Bedrock's Converse API

The Bedrock Converse API provides a consistent interface for working with all Bedrock models that support messages. This means you can write your code once and use it across different models without changes.

Key Benefits:
- Universal Interface: Same API structure works with Claude, Llama, Titan, and other models
- Model-Specific Parameters: Pass unique parameters when needed for specific models
- Privacy: Amazon Bedrock doesn't store any content you provide - data is only used for response generation
- Advanced Features: Built-in support for guardrails, tools/function calling, and prompt management 

In [ ]:
response = bedrock_client.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [{"text": "Say this is a test"}]
        }
    ],
    system=[{"text": "You are a concise, highly logical assistant."}],
    inferenceConfig={
        "temperature": 0.2,
        "maxTokens": 1000
    }
)

print(response['output']['message']['content'][0]['text'])

#### Tool Use with Converse API

The Converse API supports tool calling, allowing the model to invoke external functions. Let's demonstrate this with a weather lookup function that provides weather information for different cities.


In [ ]:
def get_weather_converse(location):
    """
    Get current weather for a given location.
    This is the same mock function used across all API examples.
    
    Args:
        location (str): City and country, e.g. "Paris, France"
        
    Returns:
        dict: Weather information
    """
    # Mock weather data - same as used in OpenAI SDK example
    weather_data = {
        "Paris, France": {"temperature": "22°C", "condition": "Partly cloudy", "humidity": "65%"},
        "New York, USA": {"temperature": "18°C", "condition": "Sunny", "humidity": "45%"},
        "Tokyo, Japan": {"temperature": "25°C", "condition": "Rainy", "humidity": "80%"},
        "London, UK": {"temperature": "15°C", "condition": "Overcast", "humidity": "70%"},
        "Sydney, Australia": {"temperature": "28°C", "condition": "Clear", "humidity": "55%"}
    }
    
    return weather_data.get(location, {
        "temperature": "20°C", 
        "condition": "Data not available", 
        "humidity": "50%"
    })

# Define the tool configuration for Converse API
converse_tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "get_weather",
                "description": "Get current temperature and weather conditions for a given location.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "City and country, e.g. 'Paris, France'"
                            }
                        },
                        "required": ["location"]
                    }
                }
            }
        }
    ]
}

print("✅ Weather function and Converse API tool configuration ready!")


In [ ]:
def generate_text_with_tools(bedrock_client, model_id, tool_config, input_text):
    """
    Generates text using the Converse API with tool support. 
    Handles tool use requests and sends results back to the model.
    
    Args:
        bedrock_client: The Boto3 Bedrock runtime client
        model_id (str): The Amazon Bedrock model ID
        tool_config (dict): The tool configuration
        input_text (str): The input text
        
    Returns:
        The final response from the model
    """
    
    # Create the initial message from the user input
    messages = [{
        "role": "user",
        "content": [{"text": input_text}]
    }]
    
    # Make the initial request to the model
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        toolConfig=tool_config
    )
    
    output_message = response['output']['message']
    messages.append(output_message)
    stop_reason = response['stopReason']
    
    if stop_reason == 'tool_use':
        print("🔧 Model requested tool use...")
        
        # Process tool use requests
        tool_requests = response['output']['message']['content']
        for tool_request in tool_requests:
            if 'toolUse' in tool_request:
                tool = tool_request['toolUse']
                print(f"🔧 Calling tool: {tool['name']} with ID: {tool['toolUseId']}")
                print(f"🔧 Tool input: {tool['input']}")
                
                if tool['name'] == 'get_weather':
                    # Call our weather function
                    try:
                        result = get_weather_converse(
                            location=tool['input']['location']
                        )
                        print(f"🔧 Tool result: {result}")
                        
                        tool_result = {
                            "toolUseId": tool['toolUseId'],
                            "content": [{"json": result}]
                        }
                    except Exception as err:
                        print(f"❌ Tool error: {err}")
                        tool_result = {
                            "toolUseId": tool['toolUseId'],
                            "content": [{"text": f"Error: {str(err)}"}],
                            "status": 'error'
                        }
                    
                    # Create the tool result message
                    tool_result_message = {
                        "role": "user",
                        "content": [{"toolResult": tool_result}]
                    }
                    messages.append(tool_result_message)
        
        # Send the tool result back to the model
        response = bedrock_client.converse(
            modelId=model_id,
            messages=messages,
            toolConfig=tool_config
        )
        output_message = response['output']['message']
    
    # Return the final response
    return output_message

print("✅ Tool use function ready!")


In [ ]:
# Test the weather tool with various location queries
test_questions = [
    "What's the weather like in Paris today?",
    "Can you tell me the temperature in Tokyo?",
    "How's the weather in Sydney, Australia?",
    "What are the conditions like in New York?"
]

print("🌤️ Testing Weather Tool with Converse API")
print("=" * 50)

for i, question in enumerate(test_questions, 1):
    print(f"\n📝 Test {i}: {question}")
    print("-" * 40)
    
    try:
        # Use gpt-oss-120b model ID - update this to match your configured MODEL_ID
        response = generate_text_with_tools(
            bedrock_client, 
            "gpt-oss-120b",  # You can change this to gpt-oss-20b for faster inference
            converse_tool_config, 
            question
        )
        
        # Print the final response from the model
        print("🤖 Model response:")
        for content in response['content']:
            if 'text' in content:
                print(content['text'])
            else:
                print(json.dumps(content, indent=2))
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
    
    print()


#### What Just Happened with Converse Tool Use?

The tool use demonstration above shows the complete flow:

1. **Tool Definition**: We defined a weather tool with a clear schema describing its inputs and outputs
2. **Model Request**: The model receives a weather question and recognizes it needs to use the weather tool
3. **Tool Invocation**: The model generates a structured tool use request with the appropriate parameters  
4. **Tool Execution**: Our code intercepts this request, calls the actual weather function, and gets the result
5. **Result Return**: We send the tool result back to the model in the proper format
6. **Final Response**: The model incorporates the tool result into a natural language response

**Key Benefits:**
- **Accuracy**: Weather data comes from actual function calls, not model approximation
- **Reliability**: Tools provide deterministic, structured results every time
- **Extensibility**: You can add any custom tools your application needs (APIs, databases, etc.)

This same pattern works for any tool - from simple weather lookups to complex API integrations!


## Tool Use API Comparison

Now that you've seen the same weather function implemented across all three APIs, here's a side-by-side comparison:

| Aspect | OpenAI SDK | InvokeModel API | Converse API |
|--------|------------|-----------------|--------------|
| **Ease of Use** | ⭐⭐⭐⭐⭐ Familiar | ⭐⭐⭐ Manual | ⭐⭐⭐⭐ Consistent |
| **Tool Schema** | OpenAI `function` format | OpenAI `function` format | Bedrock `toolSpec` format |
| **Response Handling** | SDK handles parsing | Manual JSON parsing | SDK handles parsing |
| **Error Management** | Built-in error handling | Custom error handling | Built-in error handling |
| **Performance** | SDK overhead | Direct API calls | Moderate overhead |
| **Flexibility** | Limited to OpenAI patterns | Complete customization | Bedrock-specific features |
| **Migration** | Easy from OpenAI | Full custom implementation | Learn new format |

### When to Choose Each Approach:

**OpenAI SDK**: 
- ✅ Migrating from OpenAI
- ✅ Want familiar patterns
- ✅ Quick prototyping
- ❌ Need Bedrock-specific features

**InvokeModel API**:
- ✅ Maximum control needed
- ✅ Custom request processing
- ✅ Performance critical applications
- ❌ Don't want to handle parsing manually

**Converse API**:
- ✅ Multi-model compatibility
- ✅ Bedrock native features
- ✅ Guardrails integration
- ❌ Learning new API format

All three approaches produce the same results - choose based on your specific needs and preferences!

## Conclusion

You've successfully explored **three powerful ways** to interact with OpenAI's GPT-OSS models on Amazon Bedrock, including comprehensive tool use capabilities!

### What We've Accomplished

**1. OpenAI SDK Integration**
- Set up the familiar OpenAI SDK to work seamlessly with AWS Bedrock
- Leveraged existing OpenAI patterns while running on AWS infrastructure
- Demonstrated streaming responses with real-time token generation
- **Implemented function calling** using familiar OpenAI SDK patterns

**2. Direct InvokeModel API Access**
- Implemented low-level Bedrock InvokeModel API calls for maximum control
- Built custom functions for both streaming and non-streaming inference
- Measured performance metrics like time-to-first-token for streaming responses
- Gained fine-grained control over model parameters and request formatting
- **Built custom tool use handling** with manual request/response processing

**3. Bedrock Converse API**
- Explored the unified Converse API that works across all Bedrock models
- Demonstrated consistent message-based interactions regardless of underlying model
- Leveraged built-in support for system prompts and inference configuration
- **Integrated tool calling** using Bedrock's native toolSpec format

**4. Tool Use Comparison**
- Implemented the **same weather function across all three APIs** for direct comparison
- Learned the different schema formats and response handling approaches
- Understood when to choose each API based on your specific needs

### Key Benefits Achieved

✅ **Flexibility**: Three different API approaches for different use cases  
✅ **Performance**: Streaming support for improved user experience  
✅ **Familiarity**: Use existing OpenAI SDK patterns with AWS infrastructure  
✅ **Control**: Direct API access when you need fine-grained customization  
✅ **Consistency**: Universal interface that works across all Bedrock models  
✅ **Privacy**: AWS Bedrock doesn't store your data - only used for inference  
✅ **Tool Integration**: Function calling capabilities across all three approaches
✅ **Practical Comparison**: Side-by-side examples using the same function

### What's Next?

You're now equipped with comprehensive knowledge to choose the right API approach for your specific use case. Whether you need:
- The **simplicity** of the OpenAI SDK
- The **control** of InvokeModel 
- The **consistency** of Converse API
- **Tool use capabilities** for external integrations

You have all the tools and examples to build powerful AI applications with external function calling on AWS Bedrock!

## Use Cases


Let's test reasoning with the Chat Completions API:

In [ ]:
def run_prompt(prompt, temperature=0.02, max_tokens=1000):
    """Helper function to run prompts with chat completions"""
    response = client.chat.completions.create(
        model=MODEL_ID, # update your model id to either 120b or 20b
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=max_tokens,
        stream=False
    )
    return response.choices[0].message.content

print("✅ Helper function ready!")

## Reasoning Use Case Examples

Testing complex reasoning and problem-solving capabilities with the Chat Completions API.

### Exponential Growth Problem

Tests compound growth calculations and iterative reasoning

In [ ]:
prompt = """Solve and verify.

A factory makes widgets. Day 1 output is 500. Each day after, output increases by 8%.
How many days until daily output first exceeds 1,000?

Return:
- Short reasoning outline (≤4 lines)
- Final answer (integer days)
- One-line self-check (plug back)."""

print(run_prompt(prompt, max_tokens=2000))

### Kinematics Problem

Tests multi-phase motion analysis and distance calculations

In [ ]:
prompt = """A car accelerates uniformly from rest to 27 m/s in 9 s, then coasts at constant speed for 60 s, then brakes uniformly to rest in 6 s.
Compute total distance traveled.

Return:
- Piecewise distances
- Sum
- One-line unit/sanity check."""

print(run_prompt(prompt, max_tokens=1000))

### Scientific Problem Solving

In [ ]:
science_prompt = """A ball is thrown vertically upward from ground level with an initial velocity of 20 m/s.
Calculate:
1. Maximum height reached
2. Time to reach maximum height  
3. Total time in the air
4. Velocity when it returns to ground level

Use g = 9.8 m/s². Show your physics reasoning and formulas."""

print("Scientific Problem Solving:")
print("=" * 50)
print(run_prompt(science_prompt, max_tokens=1000))

### System Design 

In [ ]:
# System Design Example  
system_prompt = """Design a URL shortener service like bit.ly for 100M URLs per day.
Address:
1. Database schema design
2. Encoding/decoding algorithm choice
3. Scalability considerations (caching, load balancing)
4. One potential bottleneck and solution

Keep response concise but show your reasoning."""

print("System Design Example:")
print("=" * 50)
print(run_prompt(system_prompt, max_tokens=3000))